# Extracts Data at Points

In [1]:
arcpy.ResetEnvironments()

# setup work space
import arcpy
import os
import glob
import tempfile
# Set environment settings
arcpy.env.workspace = "C:/Workspace/VTWetlands/VTWetlands/"
wd = arcpy.env.workspace + "/"
print(os.listdir(wd))
print(arcpy.env.workspace)
outfolder = arcpy.env.workspace + "/outputs/"
print(outfolder)

# random number generator and seed 
arcpy.env.randomGenerator = "99 ACM599"

# set the number of cores to distribute processing to
arcpy.env.parallelProcessingFactor = "50%"

# output coordinate system
template_raster = r"D:\GeoData\LandLandcov_BaseLC2016\LandLandcov_BaseLC2016.tif"
spatial_ref = arcpy.Describe(template_raster).spatialReference
arcpy.env.outputCoordinateSystem = spatial_ref 

# output cell size
# Set the cell size environment using a number.
arcpy.env.cellSize = template_raster
# Set the cell size environment using a number.
arcpy.env.cellSize = 3 # meters

import pandas as pd 
import numpy as np
from shapely.geometry import Point
from arcgis.geometry import Geometry
import math


# read in list of selected sites and shuffle values
df = pd.read_csv(wd+"sites/selected_sites_2021-03-04.csv")
roster = pd.read_csv(wd+"sites/roster.csv")
netid = roster.NETID
nstudents = len(netid)
print(netid)
print(df)



['.backups', '.ipynb_checkpoints', '.pyHistory', 'AssignRandomPoints2KMLs.ipynb', 'backup.aprx', 'ImportLog', 'Index', 'LandCoverClassification_SamplingPlots.gdb', 'LandCoverClassification_SamplingPlots.lyrx', 'LandLandcov_ImpervSrfcs2016', 'Map1.pdf', 'Map1_CDL19.pdf', 'Map1_Imagery.pdf', 'Map_image.pdf', 'Map_wetland.pdf', 'New Notebook.ipynb', 'outputs', 'sites', 'temp', 'VSWI_Wetlands_Advisory_Layer', 'VSWI_Wetlands_Advisory_Layer.zip', 'VSWI_Wetlands_Class_Layer', 'VSWI_Wetlands_Class_Layer.zip', 'VTWetlands.aprx', 'VTWetlands.aprx.xml', 'VTWetlands.gdb', 'VTWetlands.tbx', 'VTWetlands_2021-03-04.aprx', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.zip']
C:/Workspace/VTWetlands/VTWetlands
C:/Workspace/VTWetlands/VTWetlands/outputs/
0      sralger
1     bbalpard
2     nebingha
3     sbradle2
4     lbrown28
        ...   
72      extra1
73      extra2
74      extra3
75      extra4
76      extra5

In [2]:
def globsearchinpath(inpath,globsearch):
    os.chdir(inpath)
    return(glob.glob(globsearch))

def os_cmd_ogr2ogr (inpath, outpath, outformat):
    cmd = "ogr2ogr -f \"{outformat}\" {outpath} {inpath}".format(outformat=outformat,
                                                                 outpath=outpath,
                                                                 inpath=inpath)
    print(cmd)
    try:
        os.system(cmd)
        print("success")
    except:
        print("error")
    return()

# extract values from list of rasters at points
def arcpy_globExtractMultiValues2Points(inpath,globsearch,inpoints):
    os.chdir(inpath)
    inrasters = glob.glob(globsearch)
    print(inrasters)
    return(arcpy.sa.ExtractMultiValuesToPoints(inpoints,inrasters))

## Extract  Data at Points 

In [4]:
# outfile names
outnames = ['vtss_hygr',
'lclu16_wet',
'lclu16_shr',
'lclu16_agr',
'rcpp_score',
'vswi_class',
'vswi_advis']

# value_fields
invarnames = [
    'HyGrNum',
    'Class_name',
    'Class_name',
    'Class_name',
    'RCPP_SCORE',
    'CLASS',
    'CLASS']

pathnames = [r'D:\GeoData\VT_Data_-_NRCS_Soil_Survey_Units-shp\VT_Data_-_NRCS_Soil_Survey_Units.shp',
r'D:\GeoData\LandLandcov_Wetlands2016\LandLandcov_Wetlands2016.gdb\Land_Wetlands2016_poly',
r'D:\GeoData\LandLandcov_Shrublands2016\LandLandcov_Shrublands2016.gdb\Land_Shrublands2016_poly',
r'D:\GeoData\LandLandcov_Agriculture2016\LandLandcov_Agriculture2016.gdb\Land_Agriculture2016_poly',
r'D:\GeoData\VSWI\Wetland_Restoration_Model_Site_Prioritization__Lake_Champlain_2017_-shp\Wetland_Restoration_Model_Site_Prioritization__Lake_Champlain_2017_.shp',
r'D:\GeoData\VSWI\VSWI_Wetlands_Class_Layer-shp\VSWI_Wetlands_Class_Layer.shp',
r'D:\GeoData\VSWI\VSWI_Wetlands_Advisory_Layer-shp\VSWI_Wetlands_Advisory_Layer.shp']

for i in range(len(pathnames)):
    print(i)
    in_features=pathnames[i]
    value_field=invarnames[i]
    out_rasterdataset=outfolder+outnames[i]+".tif"
    print(out_rasterdataset)
    try:
        arcpy.conversion.PolygonToRaster(in_features,
                                     value_field,
                                     out_rasterdataset,
                                     cell_assignment="MAXIMUM_COMBINED_AREA")
                                     #build_rat = "BUILD")
        print("task complete",os.path.exists(out_rasterdataset))
    except:
        print("task failed\n outpath exists:",os.path.exists(out_rasterdataset))

0
C:/Workspace/VTWetlands/VTWetlands/outputs/vtss_hygr.tif
task complete True
1
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_wet.tif
task complete True
2
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_shr.tif
task complete True
3
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_agr.tif
task complete True
4
C:/Workspace/VTWetlands/VTWetlands/outputs/rcpp_score.tif
task complete True
5
C:/Workspace/VTWetlands/VTWetlands/outputs/vswi_class.tif
task complete True
6
C:/Workspace/VTWetlands/VTWetlands/outputs/vswi_advis.tif
task complete True


In [5]:
names = ["copy_selected_sites","dissolved_random"]
files = [wd+"VTWetlands.gdb/"+"copy_selected_sites",
         wd+"VTWetlands.gdb/"+"dissolved_random"]
newname="merged_extracted"
print(files)

emptyFC = arcpy.CreateFeatureclass_management(wd+"VTWetlands.gdb",
                                              newname,
                                              "POINT", 
                                              files[0])

merge = arcpy.management.Merge(files,emptyFC, "", "ADD_SOURCE_INFO")

table=arcpy_globExtractMultiValues2Points(inpath="D:/GeoData/Cropscape_VT",
                                    globsearch="*.tif",
                                    inpoints=merge)
table=arcpy_globExtractMultiValues2Points(inpath='D:\\GeoData\\LandLandcov_BaseLC2016\\',
                                    globsearch="*.tif",
                                    inpoints=merge)
table=arcpy_globExtractMultiValues2Points(inpath=outfolder,
                                    globsearch="*.tif",
                                    inpoints=merge)

arcpy.management.CopyFeatures(in_features=table,
                              out_feature_class=outfolder+newname)
    
outpath=r"C:\\Workspace\\VTWetlands\\"
outname = r"merged_extracted.csv"
if os.path.exists(os.path.join(outpath,outname)):
    print(os.path.join(outpath,outname))
    os.remove(os.path.join(outpath,outname))
arcpy.conversion.TableToTable(table,outpath,outname)
    

['C:/Workspace/VTWetlands/VTWetlands/VTWetlands.gdb/copy_selected_sites', 'C:/Workspace/VTWetlands/VTWetlands/VTWetlands.gdb/dissolved_random']
['CDL_2008_clip_20201002155541_287304339.tif', 'CDL_2009_clip_20201002155541_287304339.tif', 'CDL_2010_clip_20201002155541_287304339.tif', 'CDL_2011_clip_20201002155541_287304339.tif', 'CDL_2012_clip_20201002155541_287304339.tif', 'CDL_2013_clip_20201002155541_287304339.tif', 'CDL_2014_clip_20201002155541_287304339.tif', 'CDL_2015_clip_20201002155541_287304339.tif', 'CDL_2016_clip_20201002155541_287304339.tif', 'CDL_2017_clip_20201002155541_287304339.tif', 'CDL_2018_clip_20201002155541_287304339.tif', 'CDL_2019_clip_20201002155541_287304339.tif']
['LandLandcov_BaseLC2016.tif']
['lclu16_agr.tif', 'lclu16_imperv.tif', 'lclu16_shr.tif', 'lclu16_wet.tif', 'rcpp_score.tif', 'vswi_advis.tif', 'vswi_class.tif', 'vtss_hygr.tif']
C:\\Workspace\\VTWetlands\\merged_extracted.csv


<Result 'C:\\Workspace\\VTWetlands\\merged_extracted.csv'>